In [37]:
import os

import numpy as np

from src.data.preprocess.cxt_window import window_cxt
from src.pollution.clock_jitter import clock_jitter
from src.pollution.random_delay import random_delay
from src.trace_set.database import Database
from src.trace_set.pollution import Pollution, PollutionType
from src.trace_set.set_hw import TraceSetHW
from src.tvla.tvla import Group

In [24]:
DB_CXT = Database.aisy_raw
DB = Database.aisy

TRACE_SET_CXT = TraceSetHW(DB_CXT)
TRACE_SET = TraceSetHW(DB)

In [26]:
(X_CXT, Y), (X_ATT_CXT, Y_ATT) = TRACE_SET_CXT.profile(), TRACE_SET_CXT.attack()

In [30]:
SAMPLE_TRACE = TRACE_SET.profile()[0][0]
WINDOW, _ = window_cxt(X_CXT, SAMPLE_TRACE)

In [31]:
X = X_CXT[:, WINDOW[0]:WINDOW[1]]
X_ATT = X_ATT_CXT[:, WINDOW[0]:WINDOW[1]]

In [33]:
def fixed_fixed(x:np.ndarray, hw: np.ndarray):
    leakage_bit = hw > 4
    drop_mask = hw != 4

    return x[drop_mask], leakage_bit[drop_mask]

In [47]:
JITTER_PARAMS = np.arange(0, 2.25, .125)

def apply_jitter(params):
    for param in params:
        pollution = Pollution(PollutionType.jitter, param)
        out = TraceSetHW(DB, pollution)

        if not os.path.exists(out.path):
            x = clock_jitter(X_CXT, WINDOW, param)
            x_att = clock_jitter(X_ATT_CXT, WINDOW, param)

            out.create(x, Y, x_att, Y_ATT)
        else:
            x, _ = out.profile()

        x2, y2 = fixed_fixed(x, Y)
        a, b = x2[~y2], x2[y2]
        order = 2
        _, tvla_p = Group(a, order, True).t_test(Group(b, order, True), order)
        print(f"Jitter ({param}): min-p: ({min(tvla_p)}).")

apply_jitter([1000])

Computing Central Moments: 100%|██████████| 4/4 [00:00<00:00, 19.71it/s]

Jitter (1000): min-p: (2.739175242053216e-44).


In [ ]:
DELAY_PARAMS = np.arange(0, .75 + .125, .125)
A = 5
B = 3
DELAY_AMP = 10

def apply_delay(params):
    for param in params:
        pollution = Pollution(PollutionType.delay, param)
        out = TraceSetHW(DB, pollution)

        if not os.path.exists(out.path):
            profile_jitter = random_delay(X, A, B, DELAY_AMP, param)
            attack_jitter = random_delay(X_ATT,A, B, DELAY_AMP, param)

            out.create(profile_jitter, Y, attack_jitter, Y_ATT)